<a href="https://colab.research.google.com/github/CTEIIU/GaSDP_lab/blob/main/lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fitdecode

In [ ]:
f1 = open('sample_data/2_5303291541106735851.fit','r')

import fitdecode
import pandas as pd
import os
from datetime import datetime, timedelta
from typing import Dict, Union, Optional, Tuple
from pyproj import CRS
import geopandas as gpd
from shapely.geometry import LineString
import folium
import altair as alt
record = ['latitube', 'longitube', 'altitube','timestamp','heart_rate','cadence','speed', 'distance']
def get_fit_point_data(frame: fitdecode.records.FitDataMessage) -> Optional[Dict[str, Union[float, int, str, datetime]]]:
    data: Dict[str, Union[float, int, str, datetime]] = {}

    if not (frame.has_field('position_lat') and frame.has_field('position_long')):
        return None
    else:
        data['latitube'] = frame.get_value('position_lat') / ((2 ** 32) / 360)
        data['longitube'] = frame.get_value('position_long') / ((2**32) / 360)

    for field in record[2:]:
        if frame.has_field(field):
            data[field] = frame.get_value(field)

    return data
def get_fit_other_data(col, frame: fitdecode.records.FitDataMessage) -> Optional[Dict[str, Union[float, int, str, datetime]]]:
    data: Dict[str, Union[float, int, str, datetime]] = {}

    for field in col:
        if frame.has_field(field):
            data[field] - frame.get_value(field)
    return data

record_data = []
with fitdecode.FitReader('sample_data/2_5303291541106735851.fit') as fit_file:
    for frame in fit_file:
        if isinstance(frame, fitdecode.records.FitDataMessage):
            if frame.name == 'record':
                single_point_data = get_fit_point_data(frame)
                if single_point_data is not None:
                    record_data.append(single_point_data)

record_df = pd.DataFrame(record_data, columns=record)
line = gpd.GeoDataFrame(crs = CRS('EPSG:4326'), geometry=[LineString(zip(record_df.longitube, record_df.latitube))])
line_centroid = line.centroid
line_bounds = line.bounds

m = folium.Map([line_centroid.y, line_centroid.x], tiles='cartodb positron')
folium.GeoJson(line).add_to(m)
folium.FitBounds([[line_bounds.miny[0], line_bounds.minx[0]],[line_bounds.maxy[0],line_bounds.maxx[0]]]).add_to(m)
m

alt.Chart(record_df).mark_point().encode(y='cadence', x='distance', color='heart rate').interactive